# Example of optimizing a convex function

# Goal is to test the objective values found by Random

# Benchmarking test with different iterations for serial executions

In [1]:
from mango.tuner import Tuner
from scipy.stats import uniform

def get_param_dict():
    param_dict = {"learning_rate": uniform(0, 1),
              "gamma": uniform(0, 5),
              "max_depth": range(1,10),
              "n_estimators": range(1,300),
               "booster":['gbtree','gblinear','dart']
             }
    return param_dict

from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from sklearn.datasets import load_wine

X, Y = load_wine(return_X_y=True)

count_called = 1

def objfunc(args_list):
    global X, Y, count_called
    
    #print('count_called:',count_called)
    
    count_called = count_called + 1
    results = []
    for hyper_par in args_list:
        clf = XGBClassifier(**hyper_par)
        result = cross_val_score(clf, X, Y, scoring='accuracy').mean()
        results.append(result)
    return results

def get_conf():
    conf = dict()
    conf['batch_size'] = 1
    conf['num_iteration'] = 100
    conf['optimizer'] = "Random"
    
    return conf

def get_optimal_x():
    param_dict = get_param_dict()
    conf = get_conf()
    tuner = Tuner(param_dict, objfunc,conf)
    results = tuner.maximize()
    return results

In [2]:
Store_Optimal_X = []
Store_Results = []
num_of_tries = 20

In [3]:
for i in range(num_of_tries):
    results = get_optimal_x()
    Store_Results.append(results)
    #print(results)
    print(i,":",results['best_objective'])

0 : 0.9220306513409962
1 : 0.9164750957854406
2 : 0.9164750957854406
3 : 0.9275862068965517
4 : 0.9220306513409962
5 : 0.9220306513409962
6 : 0.9220306513409962
7 : 0.9164750957854406
8 : 0.9164750957854406
9 : 0.9220306513409962
10 : 0.9164750957854406
11 : 0.9275862068965517
12 : 0.9331417624521072
13 : 0.9275862068965517
14 : 0.9220306513409962
15 : 0.9164750957854406
16 : 0.9164750957854406
17 : 0.9331417624521072
18 : 0.9164750957854406
19 : 0.9275862068965517


# Extract from the results returned the true optimal values for each iteration

In [11]:
import numpy as np

total_experiments = 20

initial_random = 5

plotting_itr =[10, 20,30,40,50,60,70,80,90,100]

plotting_list = []

for exp in range(total_experiments): #for all exp
    local_list = []
    for itr in plotting_itr: # for all points to plot
        # find the value of optimal parameters in itr+ initial_random
        max_value = np.array(Store_Results[exp]['objective_values'][:itr+initial_random]).max()
        local_list.append(max_value)
    plotting_list.append(local_list)

In [12]:
plotting_array = np.array(plotting_list)

In [13]:
Y = []

range_min = -100
range_max = 100
#count range between -1 and 1 and show it
for i in range(len(plotting_itr)):
    y_value = plotting_array[:,i].mean()
    Y.append(y_value)

In [14]:
# Printing Y Values

In [15]:
Y

[0.9142241379310345,
 0.9164750957854407,
 0.9184195402298851,
 0.9189655172413793,
 0.9189655172413793,
 0.9195306513409962,
 0.9198084291187738,
 0.9217528735632184,
 0.922030651340996,
 0.922030651340996]

In [9]:
plotting_itr

[10, 20, 30, 40, 50, 60, 70, 80, 90, 100]

In [10]:
import numpy as np
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(10,10))


plt.plot(plotting_itr,Y,label = 'Random',linewidth=3.0) #x, y


plt.xlabel('Number of Iterations',fontsize=25)
plt.ylabel('Mean best accuracy achieved',fontsize=25)
plt.title('Variation of Optimal Value of X with iterations',fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.grid(True)
plt.yticks(np.arange(0.915, 0.925, step=0.005))
plt.legend(fontsize=20)
plt.show()

<Figure size 1000x1000 with 1 Axes>